In [1]:
import numpy as np
import pandas as pd
import numpy_financial as npf
import time
import sys
import os
import json
sys.path.append('../')

from utils.data_generation.hedge_item_gen import credit_fv_delta,CreditGenerator
from utils.data_generation.hedge_instrument_gen import SwapGenerator
from utils.solvers.mip_drop_cplex import solve_with_cplex
from utils.solvers.mip_drop_cbc import solve_with_cbc
from utils.validation_functions.validation_cplex import validate_solution_cplex
from utils.validation_functions.validation_cbc import validate_solution_cbc

pd.set_option('display.float_format', lambda x: '{:,.3f}'.format(x))


In [2]:

# scenarios
DISTRIBUTION_SCENARIOS = {
    "Cash_90-8-2": [0.90, 0.08, 0.02],
    "Mortgage_74-24-2": [0.74, 0.24, 0.02],
}

RATE_CHANGE_SCENARIOS = {
    "one_perc_change": 0.01,
    "two_perc_change": 0.02,
}

CREDIT_TYPES = ['Cash Loan Credit', 'Car Loan Credit', 'Mortgage Credit']
PRINCIPAL_RANGES = [[10_000, 50_000], [20_000, 70_000], [30_000, 100_000]]
MATURITY_RANGES  = [[4, 8], [3, 9], [2, 12]]
CREDIT_SPREAD_RANGES = [[0.005, 0.015], [0.015, 0.030], [0.020, 0.040]]

def make_experiment_name(dist_key, num_credits, num_swaps, rate_key):
    return f"{dist_key}|{rate_key}|{num_credits}_Credits|{num_swaps}_Swaps"

def generate_and_save_instance(rate_key, dist_key, num_credits, num_swaps,
                               random_seed_list, fulfillment_ratio=0.6, data_root="../data"):
    dr = RATE_CHANGE_SCENARIOS[rate_key]
    distributions = DISTRIBUTION_SCENARIOS[dist_key]
    interest_rates = [0.07, 0.07 + dr]

    credit_generator_inputs = {
        'credit_types': CREDIT_TYPES,
        'principals_ranges': PRINCIPAL_RANGES,
        'maturities_ranges': MATURITY_RANGES,
        'distributions': distributions,
        'credit_spread_ranges': CREDIT_SPREAD_RANGES,
        'num_credits': num_credits,
        'interest_rates': interest_rates
    }

    # create one folder for this setup
    experiment_name = make_experiment_name(dist_key, num_credits, num_swaps, rate_key)
    out_dir = os.path.join(data_root, experiment_name)
    os.makedirs(out_dir, exist_ok=True)

    for seed in random_seed_list:
        np.random.seed(seed)
        credit_generator = CreditGenerator(**credit_generator_inputs, random_seed=seed)
        credits_df = credit_generator.generate_credits()
        swaps_df, credits_df_v1 = SwapGenerator(credits_df, num_swaps, fulfillment_ratio, random_factor=0.95)

        credits_df_v1.to_csv(os.path.join(out_dir, f"credits_{seed}.csv"), index=False)
        swaps_df.to_csv(os.path.join(out_dir, f"swaps_{seed}.csv"), index=False)

# ---- driver ----
if __name__ == "__main__":
    num_credits_list = [50_000, 100_000, 500_000]
    num_swaps_list   = [2, 4, 8]
    rate_keys        = ["one_perc_change", "two_perc_change"]
    dist_keys        = ["Cash_90-8-2", "Mortgage_74-24-2"]
    random_seed_list = range(1, 6)

    for num_credits in num_credits_list:
        for num_swaps in num_swaps_list:
            for rate_key in rate_keys:
                for dist_key in dist_keys:
                    generate_and_save_instance(rate_key, dist_key, num_credits, num_swaps, random_seed_list)


In [7]:
valid_cred_df = pd.read_csv(os.path.join('../data/Cash_90-8-2|one_perc_change|50000_Credits|2_Swaps/', f"credits_{1}.csv"))
valid_swaps_df = pd.read_csv(os.path.join('../data/Cash_90-8-2|one_perc_change|50000_Credits|2_Swaps/', f"swaps_{1}.csv"))

In [8]:
valid_cred_df.head()

,Type,Principal,Maturity,Credit_Spread,Delta_FV,Swap,UNIQUE_INDEX
0,Cash Loan Credit,27653,4,0.011,-320.680,2,0
1,Cash Loan Credit,14154,6,0.010,-248.960,1,1
2,Cash Loan Credit,30800,7,0.014,-609.020,1,2
3,Car Loan Credit,21475,5,0.022,-305.220,2,3
4,Cash Loan Credit,39146,7,0.013,-775.130,1,4


In [9]:
valid_swaps_df.head()

,Swap,Principal,Delta_FV,Maturity
0,1,"475,321,846.000","7,186,146.474",5.550
1,2,"480,730,799.000","7,279,766.058",5.550
